In [1]:
import os
import cv2, torch
import numpy as np
import torch.nn as nn
from torch.utils.data import dataset
from torchvision import transforms
from torch.autograd import Variable

os.chdir('Y:\\Masters_Content\\Intro to AI\\Project _1\\FER_PLUS\\Codes')

In [2]:
class_dict = {
    0: 'neutral',
    1: 'happiness',
    2: 'surprise',
    3: 'sadness',
    4: 'anger'
}

#disgust,fear,contempt


In [3]:
class FERPlusInferenceDataset(dataset.Dataset):
    def __init__(self, fer_data_dir, transforms=None):
        """
        Inferencer dataset
        :param fer_data_dir: Path to images to inference (FER2013Test folder or own dataset!)
        :param transforms: test transforms
        """
        self.img_dir = fer_data_dir
        self.transforms = transforms
        self.image_file_names = sorted(
            [x for x in os.listdir(self.img_dir) if x.endswith('.jpg')])

    def __getitem__(self, idx):
        img_file_name = self.image_file_names[idx]
        img_file = os.path.join(self.img_dir, img_file_name)
        img = Image.open(img_file).convert('LA').convert('RGB')

        if self.transforms is not None:
            img = self.transforms(img)

        return img, img_file

    def __len__(self):
        return len(self.image_file_names)

In [4]:
# importing needed modules-----------------------


# defining important funtions---------------------

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# function to turn photos to tensor
loader = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# the model for predicting
model = torch.load('model_86.pth',map_location=torch.device('cpu'))


if __name__ == "__main__":
    vid = cv2.VideoCapture(0)

    while vid.isOpened():
        _, frame = vid.read()

        # takes in a gray coloured filter of the frame
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # initializing the haarcascade face detector
        faces = face_cascade.detectMultiScale(frame)
        for (x,y,w,h) in faces:
            cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)

            # takes the region of interest of the face only in gray
            roi_gray = gray[y:y+h, x:x+h]
            roi_color = frame[y:y+h, x:x+h]

            # predict the mood
            image = loader(roi_color).float()
            image = Variable(image, requires_grad=True)
            image = image.unsqueeze(0)
            outputs = model(image)
            _, pred = torch.max(outputs, 1)

            cv2.putText(frame, class_dict[pred.numpy()[0]], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0))


        cv2.imshow('video', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    vid.release()
    cv2.destroyAllWindows()